## Minimal Ollama Showcase


Quick sanity checks plus two ways to call the same JSON-focused model.

### 0. Confirm the local server is alive

In [24]:
!curl http://localhost:11434/

Ollama is running

In [25]:
!pip install requests -q

In [26]:
import json
import requests

tags = requests.get('http://localhost:11434/api/tags', timeout=30)
tags.raise_for_status()
models = [m['name'] for m in tags.json().get('models', [])]
print('Available models:', ', '.join(models) or 'none')
if any(m == 'kodzero-shorts-json:latest' for m in models):
    print('Model ready.')
else:
    print("Run 'ollama create' for kodzero-shorts-json first.")

Available models: kodzero-shorts-json:latest, kodzero-shorts:latest, qwen2.5:7b-instruct-q4_K_M, gemma2:9b-instruct-q4_K_M, llama3.1:8b-instruct-q4_K_M, SpeakLeash/bielik-7b-instruct-v0.1-gguf:Q4_K_S, deepseek-r1:7b
Model ready.


### 1. Plain requests call

In [27]:
url = 'http://localhost:11434/api/generate'
payload = {
    'model': 'kodzero-shorts-json:latest',
    'prompt': 'This is a transcript about installing Docker on a server...',
    'stream': False,
    'format': 'json'
}
raw_data = requests.post(url, json=payload, timeout=560).json()

show without context (bc too long)


In [28]:
raw_data = {k: raw_data[k] for k in set(list(raw_data.keys())) - set(['context'])}
raw_data

{'eval_duration': 33248219157,
 'done': True,
 'prompt_eval_count': 810,
 'response': '{\n  "tytul": "Docker na serwerze - instalacja i konfiguracja #shorts #viralshorts",\n  "opis": "Oto jak zainstalować Docker na swoim serwerze.\\n\\ndocker,\\nserwer,\\ninstalacja,\\nkonfiguracja,\\nkontener,\\nkontenery,\\nrepozytorium,\\napt-get,\\nupdate,\\nupgrade,\\n\\n#shorts #viralshorts #docker #serwer #linux",\n  "tagi": "docker instalacja konfiguracja serwer kontener kontenery repozytorium apt-get update upgrade",\n  "kiedy": "2/3",\n  "miniaturka": "server: datacenter; hero: sysadmin; anchors: docker logo, terminal; overlay: Polish Linux server; callouts: none; styling: vertical portrait, rule of thirds"\n}',
 'done_reason': 'stop',
 'created_at': '2025-11-15T18:37:53.12462667Z',
 'eval_count': 191,
 'load_duration': 3551659599,
 'prompt_eval_duration': 18699723206,
 'model': 'kodzero-shorts-json:latest',
 'total_duration': 86569678614}

In [29]:
final_data = json.loads(raw_data['response'])
final_data

{'tytul': 'Docker na serwerze - instalacja i konfiguracja #shorts #viralshorts',
 'opis': 'Oto jak zainstalować Docker na swoim serwerze.\n\ndocker,\nserwer,\ninstalacja,\nkonfiguracja,\nkontener,\nkontenery,\nrepozytorium,\napt-get,\nupdate,\nupgrade,\n\n#shorts #viralshorts #docker #serwer #linux',
 'tagi': 'docker instalacja konfiguracja serwer kontener kontenery repozytorium apt-get update upgrade',
 'kiedy': '2/3',
 'miniaturka': 'server: datacenter; hero: sysadmin; anchors: docker logo, terminal; overlay: Polish Linux server; callouts: none; styling: vertical portrait, rule of thirds'}

In [30]:
final_data['miniaturka']

'server: datacenter; hero: sysadmin; anchors: docker logo, terminal; overlay: Polish Linux server; callouts: none; styling: vertical portrait, rule of thirds'

### 2. Same idea with the ollama helper

In [31]:
!pip install ollama -q

In [32]:
import ollama

lib_response = ollama.generate(
    model='kodzero-shorts-json:latest',
    prompt='This is a transcript about installing Docker on a server...'
)
json.loads(lib_response['response'])

{'tytul': 'Docker na serwerze - instalacja i konfiguracja',
 'opis': 'Oto jak zainstalować i skonfigurować Docker na swoim serwerze.\n\ndocker,\nserwer,\nlinux,\nkonteneryzacja,\nkomunikacja sieciowa,\nporty,\nautoryzacja,\n\n#shorts #viralshorts #docker #serwer #linux',
 'tagi': 'docker instalacja serwer linux konteneryzacja komunikacja sieciowa porty autoryzacja',
 'kiedy': '2/3',
 'miniaturka': 'server room: dark; hero: sysadmin; anchors: docker logo, terminal; overlay: Polish Linux; callouts: none; styling: vertical portrait, high contrast'}

# BONUS

In [33]:
from ollama import chat

stream = chat(
    model='qwen2.5:7b-instruct-q4_K_M',
    messages=[{'role': 'user', 'content': 'Why is the sky blue? Answer in POLISH'}],
    stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

Kolor nieba jako zielono-szare lub niebieskie zależy od wielu czynników, ale najważniejsze jest to, jak światło słońca interluuuuuuduje z atmosferą Ziemi. Gwiezdne światło składa się z różnych barw spectry, które przenoszą się na different wavelengths. Kiedy oświetlenie trafia na molekuły w atmosferze, takie jak węgiel kohaliczny lub wodór, część tego światła jest reflektowana i rozproszona. Ta procesy rozpraszaja czerwone barwy, które mają dłuższe długości falowe, a niebieskie i zielone, które mają krótsze długości falowe, co wpływa na to, czym jest kolor nieba w danej chwili. Dlatego powietrze wydaje się być głównie niebieskie.

v1: Kolor nieba zielono-niebieski to rezultatem przenoszenia światła słońca przez atmosferę Ziemi. Przeciwko cząsteczkom w powietrzu, np. gazom i pyłowi, czerwone i żółte fale światła są odbijane silniej niż niebieskie lub بنفسowe. Dzięki temu większość niebieskiego i بنفسowego światła przenika do naszych oczu, co sprawia, że widzimy niebo jako niebieskie.

v2:
Kolor nieba jako zielono-szare lub niebieskie zależy od wielu czynników, ale najważniejsze jest to, jak światło słońca interluuuuuuduje z atmosferą Ziemi. Gwiezdne światło składa się z różnych barw spectry, które przenoszą się na different wavelengths. Kiedy oświetlenie trafia na molekuły w atmosferze, takie jak węgiel kohaliczny lub wodór, część tego światła jest reflektowana i rozproszona. Ta procesy rozpraszaja czerwone barwy, które mają dłuższe długości falowe, a niebieskie i zielone, które mają krótsze długości falowe, co wpływa na to, czym jest kolor nieba w danej chwili. Dlatego powietrze wydaje się być głównie niebieskie.